# Parsing: CKY Algorithm for PCFG
Before begining this short tutorial, we advise students to go through the pen-paper exercises in Lecture 6 which can be found in the following link https://github.com/ImperialNLP/NLPLabs/blob/master/lab05/lab05_ParsingLab_Questions.pdf. Detailed step-by-step solution to these exercises will be uploaded the following week.

In this tutorial, we will implement the [Cocke–Kasami-Younger (CKY) algorithm](https://en.wikipedia.org/wiki/CYK_algorithm) to retrieve the best parse tree of a sentence for a given (toy) version of a Probabilistic Context Free Grammar (PCFG).

We will,


1.   Understand PCFG toy-version
2.   Implement CKY Algorithm
3.   Experiment with different PCFGs



## 1. Probabilistic Context Free Grammar (PCFG)
PCFG is simply an extension of a Context Free Grammar (CFG) with probability of each rule. More formally,

$PCFG = (T, N, S, R, q)$ where


*   $T$ is set of terminals
*   $N$ is set of non-terminals
*   $S$ is start symbol
*   $R$ is a set of rules of the form $X \rightarrow Y Z$ where $X$ is a non-terminal and, $Y$ and $Z$ could be terminals or non-terminals (NOTE: we assume the rules are in [Chomsky Normal Form](https://en.wikipedia.org/wiki/Chomsky_normal_form) where there are only upto two symbols on right side of arrow $\rightarrow$)
*   $q$ is the collection of probability $P(r)$ of each rule $r \in R$ such that given a non-terminal $X$ the sum of probabilities of all rules starting with $X$ is 1. In other words, $\sum_{Y,Z}^{} P(X \rightarrow Y Z) = 1$

We estimate the probability of a rule using counts from a training set of parse trees as follows

$$P(\tilde{X} \rightarrow \tilde{Y} \tilde{Z}) = \frac{count(\tilde{X} \rightarrow \tilde{Y} \tilde{Z})}{\sum_{Y,Z} count(\tilde{X} \rightarrow Y Z)}$$

In this tutorial, we will work with a simple toy version of a PCFG (the probabilities are made up) as given below and stored in the variable ```grammar```.

```grammar``` is a list of rules where each rule is a list of four elements. The first element is the probability of the rule. The second element is the parent node of the rule. The third and fourth elements are the child nodes of the rule. For unary rules (one child only), the fourth element is ```None```.

In [0]:
grammar = [[1.0, 'S ', 'NP', 'VP'],
           [0.3, 'VP', 'V ', 'NP'],
           [0.7, 'VP', 'VP', 'PP'],
           [1.0, 'PP', 'P ', 'NP'],
           [1.0, 'P ', 'with', None],
           [1.0, 'V ', 'saw', None],
           [0.1, 'NP', 'NP', 'PP'],
           [0.2, 'NP', 'astronomers', None],
           [0.08, 'NP', 'ears', None],
           [0.15, 'NP', 'saw', None],
           [0.17, 'NP', 'stars', None],
           [0.3, 'NP', 'telescope', None]
           ]

def _print_grammar(_grammar):
  for rule in _grammar:
    if rule[3] != None:
      print(rule[1], '-->', rule[2], rule[3], '\t(p=%s)' % rule[0])
    else:
      print(rule[1], '-->', rule[2], '\t(p=%s)' % rule[0])

_print_grammar(grammar)

**Q:** Write a simple function ```_check_rule_in_grammar(_grammar, left_child, right_child)``` which checks if there exists a rule in a PCFG ```_grammar``` such that the child nodes of the rule are ```left_child``` and ```right_child```.

In [0]:
def _check_rule_in_grammar(_grammar, left_child, right_child):
  #########
  # ENTER CODE HERE
  #########

print(_check_rule_in_grammar(grammar, 'P ', 'NP'))  # Should return True
print(_check_rule_in_grammar(grammar, 'P ', 'P '))   # Should return False

# 2. Cocke–Kasami-Younger (CKY) Algorithm

CKY algorithm is a bottom-up breadth first algorithm which you must have gone through in exercise 1 of https://github.com/ImperialNLP/NLPLabs/blob/master/lab05/lab05_ParsingLab_Questions.pdf. Exercise 4 in the same document is an example of a dynamic programming version of CKY algorithm for PCFG to retrieve the most probable parse tree of a given sentence. It is not very different from the Viterbi algorithm we encountered in the previous tutorial. Pseudocode can be found in the lecture slides.

**Q:** Implement a viterbi-like Probabilistic CKY Algorithm using the provided PCFG. Complete the code below. Look out for comments ```# ENTER CODE HERE```. What is the most probable parse tree of the sentence ***astronomers saw stars with telescope***? What is its probability?  

In [0]:
import numpy as np  # numpy is not used in the Probabilistic CKY algorithm. 
                    # It is used only to print out the parse tree.

def _probabilistic_CKY_parser(_sentence, _grammar):
  """Returns the most probable parse tree of _sentence according to _grammar"""

  splitted_sentence = _sentence.split()

  viterbi = {}   # Same as pi in the pseudocode in lecture 6 
  backpointer = {}  # Same as bp in the pseudocode in lecture 6
  # We use dictionaries for convenience
  
  
  # Initialise. Filling diagonal cells (1,1) (2,2)... (n,n)
  for i in range(len(splitted_sentence)):
    word = splitted_sentence[i]
    row = i+1
    column = i+1 
    if _check_rule_in_grammar(_grammar, word, None):
      viterbi[row] = {column: {}}
      backpointer[row] = {column: {}}
      for rule in _grammar:
        if (rule[2] == word) and (rule[3] == None):
          parent = rule[1]
          probability = rule[0]
          viterbi[row][column][parent] = probability
          backpointer[row][column][parent] = {'left': [None, None, word], 
                                              'right': [None, None, None]}
    else:
      print('Error: terminal word %s not in grammar' % word)
      break
  
  # Algorithm. Filling the rest of the table.
  # Choosing a cell (row, column) in the viterbi and backpointer tables
  for j in range(len(splitted_sentence)-1):
    add_step = j+1
    for i in range(len(splitted_sentence)-add_step):
      row = i + 1
      column = row + add_step
      viterbi[row][column] = {}
      backpointer[row][column] = {}

      # Iterating over the cells in the corresponding 
      # row and column of the cell(row, column)
      for k in range(add_step):
        left_cell_row = ## ENTER CODE HERE
        left_cell_column = ## ENTER CODE HERE
        
        right_cell_row = ## ENTER CODE HERE
        right_cell_column = ## ENTER CODE HERE
            
        # Check if a rule exists such the X -> left_cell right_cell
        if viterbi[left_cell_row][left_cell_column] and viterbi[right_cell_row][right_cell_column]: 
           for left in viterbi[left_cell_row][left_cell_column].keys():
            for right in viterbi[right_cell_row][right_cell_column].keys():
              for rule in _grammar:
                if (rule[2] == left) and (rule[3] == right):
                  parent = rule[1]
                  probability = rule[0]
                  
                  # The viterbi update 'product' is the product of which three values?
                  product = ## ENTER CODE HERE
                  
                  # update the viterbi and backpointer of the cell (row,column)       
                  if parent not in viterbi[row][column].keys():
                    viterbi[row][column][parent] = product
                    backpointer[row][column][parent] = {'left': [left_cell_row, left_cell_column, left],
                                                        'right': [right_cell_row, right_cell_column, right]}
                  # What condition should go below?
                  elif ## ENTER CODE HERE:
                    viterbi[row][column][parent] = product
                    backpointer[row][column][parent] = {'left': [left_cell_row, left_cell_column, left],
                                                        'right': [right_cell_row, right_cell_column, right]}

  # Retrieving the parse tree as a matrix (Ignore the code below or improve it if you want to)                                                     
  if 'S ' not in viterbi[1][len(splitted_sentence)].keys():
    print('UNGRAMMATICAL SENTENCE BASED ON GIVEN GRAMMAR\nCANNOT BE PARSED')
  else:
    parse_matrix = np.chararray((len(splitted_sentence), len(splitted_sentence)), itemsize=2)
    parse_matrix[:] = '__'

    def recursive_function(_row, _col, _parent):
      """A recursive function to retrieve the parse tree from backpointer"""
      if _row != None:
        parse_matrix[_row-1][_col-1] = _parent

        _left_row = backpointer[_row][_col][_parent]['left'][0]
        _left_col = backpointer[_row][_col][_parent]['left'][1]
        _left_parent = backpointer[_row][_col][_parent]['left'][2]
        recursive_function(_left_row, _left_col, _left_parent)

        _right_row = backpointer[_row][_col][_parent]['right'][0]
        _right_col = backpointer[_row][_col][_parent]['right'][1]
        _right_parent = backpointer[_row][_col][_parent]['right'][2]
        recursive_function(_right_row, _right_col, _right_parent)

    recursive_function(1,len(splitted_sentence),'S ')
    
    print('Parse Tree/Matrix:\n')
    print(splitted_sentence)
    print(parse_matrix)
    print('\nProbability =', viterbi[1][len(splitted_sentence)]['S '], '\n\n')


_probabilistic_CKY_parser('astronomers saw stars with telescope', grammar)  # Should return a parse tree
_probabilistic_CKY_parser('saw stars', grammar)    # Should not return a parse tree

# 3. Experiment with different PCFGs
The probabilistic CKY parser retrieves the most probable parse tree for a given sentence according to the probabilities in the provided PCFG. If we change the probabilities in PCFG then we should get different parse trees for the same sentence.

**Q:** Create a new PCFG called ```new_grammar``` which is identical to the old PCFG ```grammar``` except for the probability scores such that we get different parse tree for the same sentence ***astronomers saw stars with telescope***. 

In [0]:
new_grammar = [[?, 'S ', 'NP', 'VP'],
               [?, 'VP', 'V ', 'NP'],
               [?, 'VP', 'VP', 'PP'],
               [?, 'PP', 'P ', 'NP'],
               [?, 'P ', 'with', None],
               [?, 'V ', 'saw', None],
               [?, 'NP', 'NP', 'PP'],
               [?, 'NP', 'astronomers', None],
               [?, 'NP', 'ears', None],
               [?, 'NP', 'saw', None],
               [?, 'NP', 'stars', None],
               [?, 'NP', 'telescope', None]
              ]

_probabilistic_CKY_parser('astronomers saw stars with telescope', new_grammar)
# This should be different from the parse tree retrieved earlier

# Doubts? Questions? Need help with this tutorial?
Contact me at clala@imperial.ac.uk or chiraag.r.lala@gmail.com :)